# ONNX Exercise: ONNX meets FastAPI

Now that you've learned about ONNX and FastAPI, let's try to use them.

Our goal: We need a REST API that predicts, whether an applicant should be granted a loan (or not). We have already trained a model for this (please run `onnx_introduction.ipynb` in before), which is available at `model/loan_model.onnx`. You are welcome to tweak the model quality, if you're keen to, but this is not subject of this exercise.

We have already started this task in the following cell, that contains part of the required code. You will find spots marked with `...`. In these spots you need to add your code.

In [ ]:
%%writefile onnx_exercise.py
import numpy as np
import onnxruntime as rt
from fastapi import FastAPI
from pydantic import BaseModel

# create the fastapi application
api = FastAPI()

# load the onnx model. for our demo purposes, the model may live in the global context.
# for more advanced apis, it may be reasonable to move the model into a dependency.
# HINT: The onnx model is supposed to be served by an InferenceSession. Check out the
# onnx_introduction.ipynb to find a use of this object.
...


# define the features that our request expects
class LoanFeatures(BaseModel):
    applicantincome: float
    ...

    @classmethod
    def get_type_fields(cls, _type) -> set:
        """Utility function to get all features of this BaseModel of a certain type.

        Example usage:
        `all_int_fields = LoanFeatures.get_type_fields(int)`  # result: {}
        """
        return {
            field for field, fieldinfo in cls.model_fields.items()
            if fieldinfo.annotation == _type
        }


# define the features that our response should send
class Prediction(BaseModel):
    ...


# create a POST endpoint that retrieves prediction requests, performs the prediction and
# returns the prediction result as its response
@api.post("/predict")
def post_predict(loan_features: LoanFeatures) -> list[Prediction]:
    # convert the request object into a dict of numpy arrays
    ins = {k: np.array(v) for k, v in loan_features.model_dump().items()}

    # convert the feature dtypes into the same dtypes that the model expects
    # NOTE that the input_value is expected to be two-dimensional, so we need to reshape it properly
    for c in LoanFeatures.get_type_fields(float):
        ins[c] = ins[c].astype(np.float32).reshape((1, 1))
    for k in LoanFeatures.get_type_fields(str):
        ins[k] = ins[k].reshape((1, 1))

    # run the prediction
    ...

    # create the response object
    # HINT: The type hint of this function's return value shows that we want to create a list of 
    # `Prediction` objects
    ...

    # return the response
    return predictions


When you execute that cell, a file called `onnx_exercise.py` will be written. You can then serve the API with the following command:

In [ ]:
!fastapi dev onnx_exercise.py --port 7777 --host 0.0.0.0

If everything worked correctly, have a look at `localhost:7777/docs`. Can you create some successful requests?